<a href="https://colab.research.google.com/github/simulate111/General/blob/main/NREL_PVWatts_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
API_KEY = "F34fZ40i424LgdCiWuwPQ9u2MF3mkWBj1NDFP76O"

locations = [
    {"city": "Copenhagen", "lat": 55.6761, "lon": 12.5683},
    {"city": "Turku",      "lat": 60.4518, "lon": 22.2666},
    {"city": "Stockholm",  "lat": 59.3293, "lon": 18.0686},
    {"city": "Oslo",       "lat": 59.9139, "lon": 10.7522}
]

url = "https://developer.nrel.gov/api/pvwatts/v8.json"

print("Fetching NREL Data (Using 'poa' with Tilt=0 for GHI)...")

for loc in locations:
    city = loc['city']
    print(f"Processing {city}...")

    # Set tilt to 0 so that Plane of Array (POA) = GHI
    params = {
        "api_key": API_KEY,
        "lat": loc['lat'],
        "lon": loc['lon'],
        "system_capacity": 1,
        "azimuth": 180,
        "tilt": 0,           # <--- CRITICAL: 0 degree tilt makes POA equal to GHI
        "array_type": 1,
        "module_type": 1,
        "losses": 14,
        "timeframe": "hourly",
        "dataset": "intl"
    }

    try:
        r = requests.get(url, params=params)

        if r.status_code == 200:
            data = r.json()

            # Check for errors returned inside a 200 OK response
            if 'errors' in data and data['errors']:
                print(f"   > API Error for {city}: {data['errors']}")
                continue

            outputs = data['outputs']

            # CORRECTION: We use 'poa' (Plane of Array) instead of 'gh'.
            # Since tilt=0, poa IS GHI.
            df = pd.DataFrame({
                'GHI_Wm2': outputs['poa'],        # Using POA as GHI
                'Air_Temperature_C': outputs['tamb'],
                'Wind_Speed_ms': outputs['wspd']
            })

            # Create TMY generic timestamps
            # NREL TMY data is always 8760 hours (non-leap year)
            df['Month'] = pd.date_range(start='2023-01-01', periods=len(df), freq='h').month
            df['Day'] = pd.date_range(start='2023-01-01', periods=len(df), freq='h').day
            df['Hour'] = pd.date_range(start='2023-01-01', periods=len(df), freq='h').hour

            # Reorder
            cols = ['Month', 'Day', 'Hour', 'Air_Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2']
            df = df[cols]

            filename = f"{city}_NREL_TMY.csv"
            df.to_csv(filename, index=False)
            print(f"   > Success! Saved {filename}")

        else:
            print(f"   > HTTP Error {r.status_code}: {r.text}")

    except Exception as e:
        print(f"   > Failed: {e}")
        # Debugging: Print available keys if we fail again
        if 'outputs' in locals():
            print(f"     Available keys were: {list(outputs.keys())}")

    time.sleep(1)

print("\nAll downloads complete.")

Fetching NREL Data (Using 'poa' with Tilt=0 for GHI)...
Processing Copenhagen...
   > Success! Saved Copenhagen_NREL_TMY.csv
Processing Turku...
   > Success! Saved Turku_NREL_TMY.csv
Processing Stockholm...
   > Success! Saved Stockholm_NREL_TMY.csv
Processing Oslo...
   > Success! Saved Oslo_NREL_TMY.csv

All downloads complete.
